In [ ]:
#处理 科目四驾考宝典1590题 科目一驾校宝典1853题
#先用word保存为docx 

In [29]:
import docx
from docx import Document
from docx.oxml import OxmlElement
from docx.enum.dml import MSO_THEME_COLOR

In [2]:
# 把有图片的题都注释上[有图片]  
# doc = Document(r'./OriginData/科目四驾考宝典1590题.docx')

# image_paragraphs = []
# for par in doc.paragraphs:
#     if 'graphicData' in par._p.xml:
#         par.text=par.text+"[有图片]"
#         #image_paragraphs.append(par)
#         #break
# doc.save('科目四驾考宝典1590题_process_pic.docx') 


#因为有些题缺失答案   在上面处理的基础上手动添加了答案 存到了：科目四驾考宝典1590题_process_pic.docx

In [30]:
#把全部文字弄成一个string
doc_path = './科目四驾考宝典1590题_process_pic.docx'

def extract_text_from_docx(doc_path):
    doc = docx.Document(doc_path)

    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    text += paragraph.text + "\n"

    return text

# 提取文档中的所有文字
all_text = extract_text_from_docx(doc_path)

In [32]:
#正则

import re
import json


pattern = r"\d+[、.](.+?)\n(.*?)答案[：:]*(.+?)\n"
#pattern = r"(\d+)[、.](.+?)\n(.*?)/n*答案：(.+?)\s"
matches = re.findall(pattern, all_text,re.DOTALL)

data = []
for match in matches:
    question = match[0].strip()
    if match[1].strip():
        options = match[1].strip()
    else :
        options="A、正确\nB、错误"

    if match[2].strip()=="√" or match[2].strip()=="正确":
        answer="A"
    elif match[2].strip()=="×" or match[2].strip()=="错误" or match[2].strip()=="错":
        answer="B"
    else:
        answer = match[2].strip()

    pic_pattern=r"[有图片]"
    if re.findall(pic_pattern, question+options+answer,re.DOTALL):
        pic_flag=True
    else:
        pic_flag=False

    
    answer_pattern=r"[^a-zA-Z]"
    if re.findall(answer_pattern, answer,re.DOTALL):
        explainnation = re.findall(r'[^a-zA-Z]+', answer)[0]
        answer=answer[0]
    else:
        explainnation=""

    question_data = {
        'question': question,
        'choose': options,
        'answer': answer,
        'picture':pic_flag,
        'explainnation':explainnation
    }

    data.append(question_data)

print("数量："+str(len(data)))

# 将数据保存为JSON文件
with open('question_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

数量：1594


In [33]:
# 过滤掉有图片的
with open('question_data.json') as file:
    data = json.load(file)
filtered_data=[item for item in data if not item['picture']]
for item in data:
    del item["picture"]

# 将数据保存为JSON文件
with open('科目四驾考宝典1590题.json', 'w', encoding='utf-8') as json_file:
    json.dump(filtered_data, json_file, ensure_ascii=False, indent=4)